## Problem 1

In this homework problem, you are going to use `tensorflow-probability` to deal with an unfair dice, i.e., a dice that has different probability of settling with each of the face 1 to 6 facing up, instead of the $p=1/6$ equal probability in the case of a fair dice.

You are provided with 5000 data entries of this dice. Each entry is a length-6 vector with one element being 1 and 0 for the rest. For example, $[0,1,0,0,0,0]$ means the "2" face of this dice landed facing up. This form is also the data form generated by `tfp.distribution.Bernoulli` when you feed multiple probability to it.

You are going to estimate the 6 probabilities describing this unfair dice (6 face). $\tilde{p} = [p1,p2,p3,p4,p5,p6]$. Keep in mind that they sum up to 1.

### Answer:
An alternate way of formulating this question is to consider the result of each of the n throws of the die as independent random variables $R$, where $R$ is distributed according to $R\sim Multinomial(n,\vec{p}) \text{ for } \vec{p}=(p_1, p_2, p_3,..., p_6)$. Then define $X_k$ to be the total number of rolls with result $k$. It can be shown that the MLE estimate for $p_k$ is given by $\hat{p}_k=x_k/n$. Calculating below...

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
tfd = tfp.distributions
tfb = tfp.bijectors

In [2]:
# Create function that allows us to write code that runs eagerly or not
def evaluate(tensors):
    if tf.executing_eagerly():
         return tf.contrib.framework.nest.pack_sequence_as(
             tensors,
             [t.numpy() if tf.contrib.framework.is_tensor(t) else t
             for t in tf.contrib.framework.nest.flatten(tensors)])
    with tf.Session() as sess:
        return sess.run(tensors)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Get the roll data, the roll sums (x_k's), and the number of rolls (n)
roll_data = tf.constant(np.loadtxt('unfair_dice.txt'), dtype=tf.float32)
roll_sums = tf.reduce_sum(roll_data, axis=0)
num_rolls = roll_data.shape[0]

# Calculate the MLE
p_hat_vec = tf.divide(roll_sums, num_rolls)

# Check that probabilities sum to 1, else adjust for rounding error
if tf.reduce_sum(p_hat_vec)==1:
    tf.print('MLE of p vector:', p_hat_vec)
else:
    p_hat_vec = tf.divide(p_hat_vec, tf.reduce_sum(p_hat_vec))
    tf.print('MLE of p vector:', p_hat_vec)
        

MLE of p vector: [0.049 0.099 0.146 0.4486 0.0508 0.2066]


### (2) Use MAP to estimate $\tilde{p}$. Selecte three different prior distribution (if the distribution is parametrized, select three different enough parameters). Using 5000 sample, compare which prior gives the best estimation of why.

[Hint: If the optimization takes too long, try to run a certain amount of steps instead of setting a criteria for the gradient. Check the remaining gradient and determine whether to increase the number of steps]

### Answer:
An alternate way to obtain a point estimate for $\vec{p}$ is using MAP, which is more akin to the Bayesian approach to creating point estimates in that it multiplies the likelyhood by a prior distribution $P(\vec{p})$, allowing us to bake in prior beliefs about the distribution of $\vec{p}$. This is Bayesian because we are now allowing ourselves to talk about $\vec{p}$ in terms of probabilities, something that is not allowed under the frequentist perspective.

Let's play the role of a casino that has noticed a suspicous amount of 1's and 4's being rolled at a specific craps table. We suspect that some of the die at the table are each weighted to land on one of these values more often than the others. To investigate, we have analyzed hours of casino floor film to get 5000 roll results of one particular die (giving our data above), and now we want to investigate if this die is weighted to 1, 4, or is unweighted. Let's make the prior for $\vec{p}$ a Dirlechet distribution, and use three different $\alpha$ paramaterizations to reflect our three prior beleifs. We will let all elements of $\vec{\alpha}$ equal 1 for the first paramaterization (prior beleif that die is not loaded), we will let $\alpha_1=5$ with all others being 1 for the second paramaterization (prior beleif that the die is loaded to land on 1), and we will let $\alpha_4=5$ with all others being 1 for the third paramaterization.

Now we want to calculate the MAP estimate for each of these paramaterizations below. However, it is much harder to derive the argmax of the resulting function with respect to $\tilde{\vec{p}}$ of this equation analytically, so we will isntead use gradient ascent to find the maximum. Doing this below...

In [5]:
test = tfd.Multinomial(total_count=num_rolls, probs=p_hat_vec)

In [6]:
# Create a list of params to try
param_0 = np.ones(6)
param_1 = np.copy(param_0)
param_5 = np.copy(param_0)
param_1[0] = 5
param_5[3] = 5

param_0 = tf.constant(param_0, dtype=tf.float32)
param_1 = tf.constant(param_1, dtype=tf.float32)
param_5 = tf.constant(param_5, dtype=tf.float32)

param_list = [param_0, param_1, param_5]
map_est_list = []

for param in param_list:
    
    LR = 0.001
    
    # Create some values to keep track of
    logit_p_vec_est = tf.constant(np.ones(6), dtype=tf.float32)
    abs_grad = np.ones(6)
    grad_list = []
    
    # Define the dirichlet distribution with the proper params as the prior distribution
    prior_dis = tfd.Dirichlet(concentration=param)
    
    while True:
        with tf.GradientTape() as tape:
            tape.watch(logit_p_vec_est)
            
            # Define the likelyhood distribution
            lh_dis = tfd.Multinomial(total_count=num_rolls, logits=logit_p_vec_est)
            
            # Define the MAP loss function
            map_loss = (lh_dis.log_prob(value=roll_sums) + \
                        prior_dis.log_prob(value=tf.nn.softmax(logit_p_vec_est)))
            
            # Take the gradient
            grad = tape.gradient(map_loss, logit_p_vec_est)
            abs_grad = np.abs(grad.numpy())
        
        # Adjust p_vec
        logit_p_vec_est += LR*grad
        
        grad_list.append(grad.numpy())
        
        # Check for convergence
        if np.all(abs_grad<0.001):
            break

    
    p_vec_est = tf.nn.softmax(logit_p_vec_est)
    tf.print(f'p_vec_est for alpha = {param}:\n{p_vec_est}')
    map_est_list.append(p_vec_est)

p_vec_est for alpha = [1. 1. 1. 1. 1. 1.]:
[0.04900011 0.09899996 0.14599997 0.44860008 0.05079995 0.20659997]
p_vec_est for alpha = [5. 1. 1. 1. 1. 1.]:
[0.04976034 0.09892079 0.14588326 0.44824144 0.05075938 0.20643483]
p_vec_est for alpha = [1. 1. 1. 5. 1. 1.]:
[0.04896094 0.09892081 0.14588326 0.44904083 0.05075934 0.20643483]


Looking at the outputs above, we can see that using an uninformative prior returns the same esimate as the MLE. Then for the priors where we alpha to five $\alpha_1$ or $\alpha_4$, we get very slightly higher probabilities for $p_1$ and $p_4$, respectively. This is to be expected as the prior is putting higher probabilities on these elements being high, but the effect is minimal since there is so much data.  

### (3) Using Monte-Carlo sampling, generate posterial samples and estimate the 6 probabilities again.

We want to estimate the distribtion for $P(\vec{p}|D)$ by drawing samples from this distribution using MCMC. We will do this using the fact that $P(\vec{p}|D)\propto P(D|\vec{p})P(\vec{p})\propto P(D|\vec{p})$ for an uninformative prior. Since we know $P(D|\vec{p})\sim Multinomial(\vec{p})$, we are in the perfect situation to estimate the distribuiton of $P(\vec{p}|D)$ using MCMC. Below, we do this using the Hamiltonian Monte Carlo algorithm.

In [15]:
# Set a function for getting the unormalized log probability of logit_p_vec_est given data
def unnormalized_log_prob(data, logit_p_vec_est):
    dist_est = tfd.Multinomial(total_count=data.shape[0], logits=logit_p_vec_est)
    class_counts = class_counts = tf.reduce_sum(data, axis=0)
    
    return tf.constant(dist_est.log_prob(class_counts))

hmc_unnormalized_log_prob = lambda p_vec: unnormalized_log_prob(roll_data, p_vec)

# Set nessecary parameters
initial_chain_state = [tf.constant(np.ones(6), dtype=tf.float32)]
step_size = tf.constant(0.01, dtype=tf.float32)
leapfrog_steps_num = 3
sample_size = int(10e3)
burnin = int(10e3)

# Set up tfp to choose adaptive step sizes for the Hamiltonian MC 
adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=hmc_unnormalized_log_prob,
        num_leapfrog_steps=leapfrog_steps_num,
        step_size=step_size
    ),
    num_adaptation_steps=int(burnin*0.008)
)

# Create funciton to gather MCMC samples
def run_chain():
    samples_hmc = tfp.mcmc.sample_chain(
        num_results=sample_size,
        num_burnin_steps=burnin,
        current_state=initial_chain_state,
        kernel=adaptive_hmc,
        trace_fn=None
    )
    return samples_hmc


In [16]:
# Gather samples and calculate the estimated p_vec
samples_hmc = run_chain()
p_vec_est = tf.nn.softmax(tf.reduce_mean(samples_hmc, axis=1))
tf.print(f'p_vec_est: {p_vec_est}')


p_vec_est: [[0.04897422 0.09898818 0.1458871  0.44879252 0.05074805 0.20660995]]


Looking at the estiamate for our vector p above, we can see that HMC gets approximately the same answer as the previous methods.